# <font color = darkblue> Bem-vindo/a!

Este estudo foi desenvolvido pelos alunos

* Afonso Coelho (FCUP_IACD:202305085)

* Diogo Amaral (FCUP_IACD:202305187)

* Miguel Carvalho (FCUP_IACD:202305229)

no âmbito programático da Licenciatura em Inteligência Artifical e Ciência de Dados (L:IACD), em colaboração com as Faculdades de Ciências e Engenharia da Universidade do Porto (FCUP/FEUP).

### <font color = darkblue> Índice:

1. [Introdução à análise:](#intro) <br> 
    >1.1 [Contexto do problema](#contexto) <br>
    >1.2 [Expectativas e objetivos](#expect) <br>
    
2. [Inicialização do dataset](#init)

3. [Estatísticas descritivas báscias:](#estatisticas) <br>
    >3.1 [Média](#media) <br>
    >3.2 [Mediana](#mediana) <br>
    >3.3 [Desvio](#desvio) <br>
    >3.4 [Assimetria](#assimetria) <br>
    >3.5 [Curtose](#curtose) <br>
    
4. [Relotório de Análise - DataPrep](#relatorio) <br>

5. [Inputação dos missing values:](#missing-values) <br>
    >5.1 [Identificação visual dos missing values](#visual-missing-values) <br>
    >5.2 [Heterogeneous Euclidean-Overlap Metric para medir distâncias entre pacientes](#heom) <br>
    >5.3 [Inputação de missing values por HEOM](#heom-missing-values) <br>

6. [Ajuste de Outliers](#outliers) <br>
    >6.1 [Identificação visual dos outliers](#visual-outliers) <br>
    >6.2 [HEOM para enquadramento dos outliers mais distantes](#heom-outliers) <br>

7. [K-Nearest Neighbors](#knn) <br>
    >7.1 [Import de bibliotecas e inicialização de classes](#knn-class) <br>
    >7.2 [Separeção train/test e fitting do algoritmo](#knn-fit) <br>
    >7.3 [Accuracy e precision do algoritmo](#knn-acc) <br>

8. [Decision Tree](#tree) <br>
    >8.1 [Import de bibliotecas e inicialização de classes](#tree-class) <br>
    >8.2 [Separeção train/test e fitting do algoritmo](#tree-fit) <br>
    >8.3 [Accuracy e precision do algoritmo](#tree-acc) <br>

9. [Logistic Regression](#log) <br>
    >9.1 [Import de bibliotecas e inicialização de classes](#log-class) <br>
    >9.2 [Separeção train/test e fitting do algoritmo](#log-fit) <br>
    >9.3 [Accuracy e precision do algoritmo](#log-acc) <br>

# <font color = darkblue> 1. Introdução à análise <a class="anchor" id="intro"></a>

### <font color = blue> 1.1 Contexto do problema <a class="anchor" id="contexto"></a>

O projeto em mãos consiste na análise de um caso real do conjunto de dados representativo de pacientes que sofrem de Carcinoma Hepatocelular (HCC), mais comumente conhecido como cancro do fígado. O conjunto mencionado de dados HCC ([hcc_dataset.csv](hcc_dataset.csv)) foi recolhido no Centro Hospitalar e Universitário de Coimbra (CHUC) em Portugal, e contém dados clínicos reais de pacientes diagnosticados com HCC.

### <font color = blue> 1.2 Expectativas e objetivos <a class="anchor" id="expect"></a>

# <font color = darkblue> 2. Inicialização do dataset <a class="anchor" id="init"></a>

In [2]:
import pandas as pd
import numpy as np
import heapq
class Dataset:
    def __init__(self, df, missing_values):
        self.df = df
        self.missing_values = missing_values


    def pintarMissingValues(self):#pintar a tabela de missing values
        if self.missing_values is not None:#se existirem missing values
            self.df.replace(self.missing_values, "NaN", inplace=True)#substituir missing values por string "NaN" devido a limitação do site 
            return self.df.style.applymap(lambda valor: "color: red;" if valor=="NaN" else "")#pintar missing values a vermelho
        else: return self.df #se não existirem missing values


    def missing_values_percentagem(self):#Percentagem de missing values
        self.df.replace(self.missing_values, np.nan, inplace=True)#substituir missing values por NaN e nao string "NaN"
        missing_values_percentages = self.df.isnull().mean() * 100#calcular a percentagem de missing values
        return missing_values_percentages.tolist()#retornar a percentagem de missing values
    
    
    def remove_int_columns(self):
        df_copy = self.df.copy()  # create a copy of the dataframe
        numerical=self.df_num()
        common_columns = set(self.df.columns) & set(numerical.columns)
        df_copy = df_copy.drop(common_columns, axis=1)
        
        return df_copy
    

    def df_num(self):
        # Replace missing values with None
        dataframe= self.replace_nan_with_none()

        # Convert all columns to numeric, replacing non-numeric values with NaN
        for col in dataframe.columns:
            dataframe[col] = pd.to_numeric(dataframe[col], errors='coerce')

        # Remove columns that only contain None values
        self.df = dataframe.dropna(axis=1, how='all')

        return self.df


    def replace_nan_with_none(self):
        self.df.replace(self.missing_values, None, inplace=True)
        return self.df


    def pintarOutliers(self, df, outliers):
        def highlight_value(series, column):#Pintar as células que são outliers de azul
            return ['background-color: blue' if (column, index) in outliers else '' for index in series.index]
        return df.style.apply(lambda x: highlight_value(x, x.name), axis=0)#Aplicar a função a cada coluna
    

    def tabelaHEOM(self):
        self.df = self.replace_nan_with_none()#Trocar missing values para none
        tabela = pd.DataFrame()
        for i in range(len(self.df)):
            lista = []
            for j in range(len(self.df)):#Não interessa comparar pares de pacientes duas vezes
                if i >= j:
                    lista.append("X")# colocar x por motivos estéticos
                else:
                    lista.append(self.HEOM(i, j))# lista de um paciente em calculo HEOM

            tabela = pd.concat([tabela, pd.DataFrame({i: lista})], axis=1)#adicionar a lista à tabela
        return tabela
    

    def HEOM(self, paciente_1, paciente_2): #Heterogeneous Euclidean-Overlap Metric
        soma = 0
        for feature in self.df.columns:# iterar sobre as V
            distancia = self.distanciaGeral(feature, paciente_1, paciente_2)# calcular a sua "distancia"
            soma += distancia**2
        soma= soma**(1/2)
        return soma
    

    def distanciaGeral(self, feature:str, paciente_1:int, paciente_2:int)->int:
        try :#Se a variavel for numerica vem para aqui
            #distancia normalizada
            valorPaciente_1 = float(self.df.loc[paciente_1, feature])
            valorPaciente_2 = float(self.df.loc[paciente_2, feature])
            numeric_feature = pd.to_numeric(self.df[feature], errors='coerce')
            return abs(valorPaciente_1 - valorPaciente_2) / (numeric_feature.max() - numeric_feature.min())# retornar a range 
        except :#Se a variavel for categorica vem para aqui
            valorPaciente_1 = self.df.loc[paciente_1, feature]
            valorPaciente_2 = self.df.loc[paciente_2, feature]
            if valorPaciente_1 == valorPaciente_2 and  not pd.isna(valorPaciente_1):#Se forem iguais e não forem missing values
                return 0
            else: 
                return 1
    

    def outliers(self,info:str,vizinhos=None):
        # Selecionar apenas as colunas numéricas
        categorical_features = self.remove_int_columns() 
        numeric_df = self.df_num()

        colunas_numericas = numeric_df.columns
        if info == 'style':
            outliers = set()
        for coluna in colunas_numericas:#calcular os outliers usando o IQR
            if info == 'tratamento':
                outliers = []
            q1 = numeric_df[coluna].quantile(0.25)
            q3 = numeric_df[coluna].quantile(0.75)
            iqr = q3 - q1
            limite_inferior = q1 - 1.5 * iqr
            limite_superior = q3 + 1.5 * iqr
            for index, value in numeric_df[coluna].items():#adicionar outliers ao set
                if value < limite_inferior or value > limite_superior:
                    if info == 'tratamento' and coluna not in ["Iron", "Sat", "Ferritin"]:
                        if self.df.loc[index, coluna] > limite_superior * 5 or self.df.loc[index, coluna] < limite_inferior * 5:
                            outliers.append((index, coluna))
                    elif info == 'style':
                        outliers.add((coluna, index))
            if info == 'tratamento':
                self.df= self.tratamentoOutliers(outliers, coluna,vizinhos)
        if info == 'style':
            # Apply styling to outliers
            styled_df = self.pintarOutliers(numeric_df, outliers)
            return styled_df
        if info == 'tratamento': 
            self.df = (pd.concat([categorical_features,self.df ], axis=1))
            return self.df
    
    
    def tratamentoOutliers(self, outliers, coluna, vizinhos):
        
        lista_valores = self.df[coluna].tolist()#todos os valores da coluna 
        contador = -1
        valores_out = [self.df.loc[index,coluna] for index,coluna in outliers]#valores dos outliers
        for valor_outlier in valores_out:# iterar por todos os outliers
            contador+=1
            outlier = valor_outlier
            dicionario_distancias = []
            for valor in lista_valores:

                if outlier != valor and valor not in valores_out and not pd.isna(valor):

                    distancia = self.HEOM(lista_valores.index(valor), lista_valores.index(outlier))#calcular a distancia entre o outlier e os outros valores
                    if len(dicionario_distancias) < vizinhos:
                        heapq.heappush(dicionario_distancias, (-distancia, valor))
                    else:
                        if -distancia > dicionario_distancias[0][0]:
                            heapq.heapreplace(dicionario_distancias, (-distancia, valor))

            k_proximos = [abs(item[1]) for item in dicionario_distancias]# selecionar os k vizinhos mais proximos
            
            media = sum(k_proximos)/len(k_proximos)
            self.df.loc[outliers[contador][0], coluna] = media
        return self.df


    def fill_missing_values(self, nr_vizinhos:int) -> pd.DataFrame:

        self.df = self.replace_nan_with_none() # Replace missing values with None 

        self.df = self.df.drop(['Iron', 'Sat', 'Ferritin'], axis=1)# Drop unnecessary columns
        df_copiada = self.df.copy()# Create a copy of the DataFrame

        for i in range(len(self.df)): # Iterate over each row
            row = self.df.iloc[i]
            
            if row.isnull().any():# Check if the row has any missing values
                closest_rows = self.linhas_mais_proximas(nr_vizinhos, i)# Get the indices of the closest rows
    
                for col in self.df.columns:# Iterate over each column
                    if pd.isnull(row[col]): # If the value is missing, replace it with the most common value or mean from the closest rows
                        df_copiada.loc[i, col] = self.subs_na_tabela(closest_rows, col,nr_vizinhos,i)
        return df_copiada
    

    def subs_na_tabela(self, closest_rows:list, col:int,vizinhos,i)->float | str :
        # Initialize values
        column_values = []

        for row_index in closest_rows:
            try:
                # Check the type of values
                value = float(self.df.loc[row_index, col])
            except:
                value = self.df.loc[row_index, col]

            if value is not None and not pd.isna(value):
                column_values.append(value)
        if len(column_values) == 0:

            return self.subs_na_tabela(self.linhas_mais_proximas(vizinhos+1,i), col,vizinhos+1,i)
        # Calculate the result based on the type of values
        if isinstance(column_values[0], str):

            # If values are strings, return the most frequent value
            return max(set(column_values), key=column_values.count)
        elif isinstance(column_values[0], (int, float)):

            # If values are numeric, return the mean
            return np.mean(column_values)
        

    def linhas_mais_proximas(self, vizinhos:int,i:int)->list: # Calculate the HEOM distance for each other row
       
        heom_values = []

        for j in range(len(self.df)):

            if j != i:
                heom_distance = self.HEOM(i, j)# Calculate the HEOM distance
                if len(heom_values) < vizinhos: # If we have less than 'vizinhos' distances, we add it to the heap

                    heapq.heappush(heom_values, (-heom_distance, j))
                else:
                    if -heom_distance > heom_values[0][0]: # If the current distance is smaller than the largest distance in the heap, we replace it
                        heapq.heapreplace(heom_values, (-heom_distance, j))
    
        # Get the rows with the smallest HEOM distance
        closest_rows = [item[1] for item in heom_values]

        return closest_rows
    

    def categorical_to_numerical(self):
        """
        _summary_: converts all categorical features to numerical values

        _conversion_dictionary_:
            Male -> 0
            Female -> 1
            No -> 0
            Yes -> 1
            Disabled -> 0
            Ambulatory -> 1
            Restricted -> 2
            Selfcare -> 3
            Active -> 4
            None -> 0
            Grade I/II -> 1
            Grade III/IV -> 2
            Mild -> 1
            Moderate/Severe -> 2
            Dies -> 0
            Lives -> 1
            
        """
        words = ("Male","Female","No","Yes","Disabled","Ambulatory",
                 "Restricted","Selfcare","Active","None","Grade I/II",
                 "Grade III/IV","Mild","Moderate/Severe","Dies","Lives")
        values = (0,1,0,1,0,1,2,3,4,0,1,2,1,2,0,1)
        self.df.replace(words, values, inplace=True)
        return self.df

    def polirTabela(self):
        self.df = self.outliers("tratamento", 3)#tratar outliers
        self.df = self.fill_missing_values(3)#preencher missing values
        self.df = self.categorical_to_numerical()#converter variaveis categoricas para numericas
        return self.df


    @classmethod #este classmethod funciona como um construtor alternativo e construir um dataframe a partir de um arquivo cs

    def builderData(cls, df, missing_values): 
        try:
            if not isinstance(df, pd.DataFrame):# Handle DataFrame input directly
                df = pd.read_csv(df)
            df = df.copy()# Avoid modifying the original DataFrame
            return cls(df, missing_values)
        except (FileNotFoundError, pd.errors.ParserError):
            # Handle potential errors: file not found or parsing errors
            print(f"Erro: Não conseguiu ler a data de {df}.")
            raise

# <font color = darkblue> 3. Estatísticas descritivas básicas <a class="anchor" id="estatisticas"></a>

### <font color = blue> 3.1. Média <a class="anchor" id="media"></a>

In [177]:
data = Dataset.builderData("hcc_dataset.csv", "?")
data = data.df
data.to_csv("hcc_dataset.csv", index=False)
tabela = data.mean(numeric_only=True).to_frame("Média")
display(tabela)

,Média
Age,64.690909
Grams_day,71.008547
Packs_year,20.464286
INR,1.421851
AFP,19299.951146
Hemoglobin,12.879012
MCV,95.119753
Leucocytes,1473.961549
Platelets,113206.442654
Albumin,3.445535


### <font color = blue> 3.2. Mediana <a class="anchor" id="mediana"></a>

In [176]:
data = Dataset.builderData("hcc_dataset.csv", "?")
data = data.df
data.to_csv("hcc_dataset.csv", index=False)
tabela = data.median(numeric_only=True).to_frame("Mediana")
display(tabela)

,Mediana
Age,66.00
Grams_day,75.00
Packs_year,0.00
INR,1.30
AFP,33.00
Hemoglobin,13.05
MCV,94.95
Leucocytes,7.20
Platelets,93000.00
Albumin,3.40


### <font color = blue> 3.3. Desvio Padrão <a class="anchor" id="desvio"></a>

In [26]:
data = Dataset.builderData("hcc_dataset.csv", "?")
data = data.df
data.to_csv("hcc_dataset.csv", index=False)
tabela = data.std(numeric_only=True).to_frame("Desvio Padrão")
display(tabela)

,Desvio Padrão
Age,13.319534
Grams_day,76.277680
Packs_year,51.565130
INR,0.477816
AFP,149098.335581
Hemoglobin,2.145237
MCV,8.405846
Leucocytes,2909.106006
Platelets,107118.632481
Albumin,0.685132


### <font color = blue> 3.4. Assimetria <a class="anchor" id="assimetria"></a>

In [28]:
data = Dataset.builderData("hcc_dataset.csv", "?")
data = data.df
data.to_csv("hcc_dataset.csv", index=False)
tabela = data.skew(numeric_only=True).to_frame("Assimetria")
display(tabela)

,Assimetria
Age,-0.779988
Grams_day,1.986666
Packs_year,7.886234
INR,3.587974
AFP,11.392308
Hemoglobin,-0.441710
MCV,-0.087315
Leucocytes,1.865177
Platelets,0.950930
Albumin,-0.102843


### <font color = blue> 3.5. Curtose <a class="anchor" id="curtose"></a>

In [178]:
data = Dataset.builderData("hcc_dataset.csv", "?")
data = data.df
data.to_csv("hcc_dataset.csv", index=False)
tabela = data.kurtosis(numeric_only=True).to_frame("Curtose")
display(tabela)

,Curtose
Age,0.846190
Grams_day,7.931784
Packs_year,74.241726
INR,18.965445
AFP,136.018560
Hemoglobin,0.402549
MCV,0.971063
Leucocytes,2.432640
Platelets,0.504059
Albumin,-0.699851


# <font color = darkblue> 4. Relatório DataPrep <a class="anchor" id="relatorio"></a>

In [192]:
def criar_e_exibir_relatorio(ficheiro):
    data_file = pd.read_csv(ficheiro, na_values='?')
    data_file.to_csv(ficheiro, index=False)
    return create_report(data_file).show_browser()
criar_e_exibir_relatorio("hcc_dataset.csv")

  0%|          | 0/7332 [00:00<?, ?it/s]

C:\Users\diogo\anaconda3\Lib\site-packages\dask\core.py:127: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
C:\Users\diogo\anaconda3\Lib\site-packages\dataprep\eda\distribution\render.py:274: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame({col: [nrows - npresent]}, index=["Others"]))
C:\Users\diogo\anaconda3\Lib\site-packages\dataprep\eda\distribution\render.py:274: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame({col: [nrows - npresent]}, index=["Others"]))
C:\Users\diogo\anaconda3\Lib\site-packages\dataprep\eda\distribution\render.py:274: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFram

Este bloco de código destina-se à criação de um relatório com base na bibloteca ``dataprep``.
É expectável encontrar informações sobre:

* Percentagem de ``missing values``
* As mais ``importantes estatísticas descritivas`` de todas as variáveis
* Distribuições de dados em ``Q-Q Plot's`` e em ``BoxPlot's``
* ``Interações`` entre 2 variáveis
* ``Correlações`` entre variáveis
* entre outros

# <font color = darkblue> 5. Inputação dos missing values <a class="anchor" id="missing-values"></a>

### <font color = blue> 5.1. Identificação visual dos missing values <a class="anchor" id="visual-missing-values"></a>

In [ ]:
data = Dataset.builderData("hcc_dataset.csv", "?")
data.pintarMissingValues()

### <font color = blue> 5.2. Heterogeneous Euclidean-Overlap Metric <a class="anchor" id="heom"></a>

In [ ]:
data = Dataset.builderData("hcc_dataset.csv", "?")
data.tabelaHEOM()

### <font color = blue> 5.3 Inputação dos missing values por HEOM <a class="anchor" id="heom-missing-values"></a>

In [ ]:
data = Dataset.builderData("hcc_dataset.csv", "?")
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
data.tratamentoMissingValues()

# <font color = darkblue> 6. Ajuste dos outliers <a class="anchor" id="outliers"></a>

### <font color = blue> 6.1. Identificação visual dos outliers <a class="anchor" id="visual-outliers"></a>

In [ ]:
data = Dataset.builderData("hcc_dataset.csv", "?")
display(data.outliers('style'))

### <font color = blue> 6.2. HEOM para enquadramento dos ouliers mais distantes <a class="anchor" id="heom-outliers"></a>

In [ ]:
data = Dataset.builderData("hcc_dataset.csv", "?")
display(data.outliers('tratamento', 3))

# <font color = darkblue> 7. K-Nearest Neighbors <a class="anchor" id="knn"></a>

### <font color = blue> 7.1. Import de bibliotecas e inicialização de classes <a class="anchor" id="knn-class"></a>

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import accuracy_score, precision_score
from sklearn.discriminant_analysis import StandardScaler

knn = KNeighborsClassifier(n_neighbors=4) #COMPLETAR VALORES OTIMOS
scaler = StandardScaler()

### <font color = blue> 7.2. Separação train/test e fitting do algoritmo <a class="anchor" id="knn-fit"></a>

In [ ]:
data = Dataset.builderData("hcc_dataset.csv", "?")
X = data.df.drop(columns=['Class']).dropna()
y = data.df['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=456) #COMPLETAR VALORES OTIMOS
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
fit = knn.fit(X_train, y_train)

### <font color = blue> 7.3. Accuracy e precision do algoritmo <a class="anchor" id="knn-acc"></a>

In [ ]:
acc = accuracy_score(y_test, knn.predict(X_test))
pre = precision_score(y_test, knn.predict(X_test))
print(f"Accuracy: {acc}, Precision: {pre}")

# <font color = darkblue> 8. Decision Tree <a class="anchor" id="tree"></a>

### <font color = blue> 8.1. Import de bibliotecas e inicialização de classes <a class="anchor" id="tree-class"></a>

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import accuracy_score, precision_score
from sklearn.discriminant_analysis import StandardScaler
clf = DecisionTreeClassifier(max_depth=depth, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf) #COMPLETAR VALORES OTIMOS
scaler = StandardScaler()

### <font color = blue> 8.2. Separação train/test e fitting do algoritmo <a class="anchor" id="tree-fit"></a>

In [ ]:
data = Dataset.builderData("hcc_dataset.csv", "?")
X = data.df.drop(columns=['Class']).dropna()
y = data.df['Class']
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=456)#COMPLETAR VALORES OTIMOS
clf.fit(X_train, y_train)

### <font color = blue> 8.3. Accuracy e precision do algoritmo <a class="anchor" id="tree-acc"></a>

In [ ]:
acc = accuracy_score(y_test, clf.predict(X_test))
pre = precision_score(y_test, clf.predict(X_test))
print(f"Accuracy: {acc}, Precision: {pre}")

# <font color = darkblue> 9. Logistic Regression <a class="anchor" id="log"></a>

### <font color = blue> 9.1. Import de bibliotecas e inicialização de classes <a class="anchor" id="log-class"></a>

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score

model = LogisticRegression(class_weight=weight) #COMPLETAR VALORES OTIMOS
scaler = StandardScaler()

### <font color = blue> 9.2. Separação train/test e fitting do algoritmo <a class="anchor" id="log-fit"></a>

In [ ]:
data = Dataset.builderData("hcc_dataset.csv", "?")
X = data.df.drop(columns=['Class']).dropna()
y = data.df['Class']
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=456)#COMPLETAR VALORES OTIMOS
model.fit(X_train, y_train)

### <font color = blue> 9.3. Accuracy e precision do algoritmo <a class="anchor" id="log-acc"></a>

In [ ]:
acc = accuracy_score(y_test, model.predict(X_test))
pre = precision_score(y_test, model.predict(X_test))
print(f"Accuracy: {acc}, Precision: {pre}")

# AINDA FALTA ADICIONAR O RANDOM TREE CASO ESTEJA FEITO (NAO ENCONTREI NA APP), INTRODUZIR OS VALORES OTIMOS ONDE DISSER "COMPLETAR VALORES OTIMOS" E CORRER O CODIGO TODO